In [1]:
import pickle
        
with open("data_extract/data_json_1.pickle","rb") as f:
    data_1 = pickle.load(f)

with open("data_extract/data_json_2.pickle","rb") as f:
    data_2 = pickle.load(f)

with open("data_extract/data_json_3.pickle","rb") as f:
    data_3 = pickle.load(f)

with open("data_extract/data_json_4.pickle","rb") as f:
    data_4 = pickle.load(f)

with open("data_extract/data_json_5.pickle","rb") as f:
    data_5 = pickle.load(f)

with open("data_extract/data_json_6.pickle","rb") as f:
    data_6 = pickle.load(f)

with open("data_extract/data_json_7.pickle","rb") as f:
    data_7 = pickle.load(f)

with open("data_extract/data_json_8.pickle","rb") as f:
    data_8 = pickle.load(f)

with open("data_extract/data_json_99.pickle","rb") as f:
    data_99 = pickle.load(f)

data = data_1 + data_2 + data_3 + data_4 + data_5 + data_6 + data_7 + data_99#35737

In [2]:
data_d = {}
for d in data:
    data_d[d["id"]]=d

In [3]:
ID_list = []
for v in data:
    ID_list.append(v["taxon_rank"])
    ID_list.append(v["parent_taxon"])
ID_set = set(ID_list)

In [4]:
ID_d = dict()
nodata_set = set()
for k in ID_set:
    if k in data_d:
        ID_d[k] = data_d[k]["en_name"]
    else:
        nodata_set.add(k)

In [5]:
ID_d_ja = dict()
nodata_set_ja = set()
for k in ID_set:
    if k in data_d:
        ID_d_ja[k] = data_d[k]["ja_name"]
    else:
        nodata_set_ja.add(k)

In [6]:
# 手動で修正

# for k in nodata_set:
#     if k == None:
#         pass
#     print(k)
#     print(k)
#     print("en_name=")
#     ID_d[k]  = input()
#     print("ja_name")
#     ID_d_ja[k] = input()

In [7]:
non_data = []
for d in data:
    try:
        d["taxon_rank_name"] = ID_d[d["taxon_rank"]]
        d["taxon_rank_ja_name"] = ID_d_ja[d["taxon_rank"]]

        d["parent_taxon_name"] = ID_d[d["parent_taxon"]]
        d["parent_taxon_ja_name"] = ID_d_ja[d["parent_taxon"]]


        s = ""
        for k,v in d["en_aliases"].items():
            if s == "":
                s = s + v
            else:
                s = s + "," +v
        d["en_aliases"] = s

        s = ""
        for k,v in d["ja_aliases"].items():
            if s == "":
                s = s + v
            else:
                s = s + "," +v
        d["ja_aliases"] = s

        s = ""
        for k,v in d["img_urls"].items():
            if s == "":
                s = s + v
            else:
                s = s + " " +v
        d["img_urls"] = s
    except:
        non_data.append(d["taxon_rank"])

In [8]:
non_data = set(non_data)

In [9]:
non_data

{None, 'Q227936', 'Q279749', 'Q34740', 'Q37517', 'Q68947', 'Q7432'}

## パスの作成，実装を試してみる

In [10]:
from anytree import Node, RenderTree

In [11]:
node_set = set()

for d in data:
    _id = d["id"].replace("]","")
    if type(d["parent_taxon"]) == str:
        _parent = d["parent_taxon"].replace("]","")
    
    if d["parent_taxon"] in node_set:
        exec('%s = Node(d["id"],parent= %s)' % (_id,_parent))
        node_set.add(d["id"])
    elif d["parent_taxon"] != None:
        exec('%s = Node(d["parent_taxon"])' % (_parent))
        node_set.add(d["parent_taxon"])
        exec('%s = Node(d["id"],parent= %s)' % (_id,_parent))
        node_set.add(d["id"])
    else:
        exec('%s = Node(d["id"],parent= None)' % (_id))

In [12]:
for d in data:
    _id = d["id"].replace("]","")
    exec('d["path"] = str(%s).replace("Node(","").replace(")","")' % (_id))

## データの書き出し

In [13]:
filedname_list = ['id','en_name','ja_name','en_aliases','ja_aliases','img_urls','taxon_name','taxon_rank','taxon_rank_name','taxon_rank_ja_name','parent_taxon','parent_taxon_name','parent_taxon_ja_name',"path"]

import csv

with open('data_extract/ontology.tsv','w',encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = filedname_list,delimiter = "\t")
    writer.writeheader()
    writer.writerows(data)

In [14]:
# data
import pickle

with open("data_extract/ontology.pickle", 'wb') as f:
    pickle.dump(data,f)